<a href="https://colab.research.google.com/github/nvaikunt/PromptBasedReranking/blob/main/BaselineTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/nvaikunt/PromptBasedReranking.git

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda --version

In [ ]:
%cd PromptBasedReranking/

In [3]:
!bash setup.sh

- \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
Solving environment: / failed with initial frozen solve. Retrying with flexible solve.


In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
!python Baseline/download_dpr_data.py --resource data.wikipedia-split.psgs_w100

Requested resource from %s https://www.dropbox.com/s/bezryc9win2bha1/psgs_w100.tar.gz
Download root_dir %s ./
File to be downloaded as %s /content/PromptBasedReranking/downloads/data/wikipedia-split/psgs_w100.tsv

Downloaded to %s /content/PromptBasedReranking/downloads/data/wikipedia-split/psgs_w100.tar.gz
Uncompressing %s /content/PromptBasedReranking/downloads/data/wikipedia-split/psgs_w100.tar.gz
 Saved to %s ./downloads/data/wikipedia-split


In [6]:
!python Baseline/download_dpr_data.py --resource data.retriever-outputs.dpr.nq-train

Requested resource from %s https://www.dropbox.com/s/6g4erof4ifg8xea/nq-train.tar.gz
Download root_dir %s ./
File to be downloaded as %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-train.json

Downloaded to %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-train.tar.gz
Uncompressing %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-train.tar.gz
 Saved to %s ./downloads/data/retriever-outputs/dpr


In [7]:
!python Baseline/download_dpr_data.py --resource data.retriever-outputs.dpr.nq-dev

Requested resource from %s https://www.dropbox.com/s/257quanu64w9sh0/nq-dev.tar.gz
Download root_dir %s ./
File to be downloaded as %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-dev.json

Downloaded to %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-dev.tar.gz
Uncompressing %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-dev.tar.gz
 Saved to %s ./downloads/data/retriever-outputs/dpr


In [8]:

import torch
from torch.utils.data import Dataset, DataLoader
import transformers
import tokenizers
import datasets 
import pandas

wikipedia_txt = datasets.load_dataset("csv", data_files='downloads/data/wikipedia-split/psgs_w100.tsv', delimiter='\t')


Using custom data configuration default-cb3c6fef8f5e82e0


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-cb3c6fef8f5e82e0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
wikipedia_txt

In [9]:
nq_open = datasets.load_dataset("json",data_files="downloads/data/retriever-outputs/dpr/nq-train.json")


Using custom data configuration default-a7ea593b34a6dc21


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-a7ea593b34a6dc21/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
nq_open["validation"] = datasets.load_dataset("json",data_files="downloads/data/retriever-outputs/dpr/nq-dev.json", split="train")

Using custom data configuration default-1626d379dd1634b3


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-1626d379dd1634b3/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


In [11]:
def get_top_k_pos(row, k, txt_database):
  ctxs = row["ctxs"]
  top_k = []
  for ctx in ctxs:
    if ctx["has_answer"]:
      text = txt_database[ctx["id"] - 1]["text"]
      top_k.append((text, "true"))
    if len(top_k) == k:
      break
  if len(top_k) == 0: 
    return []
  while len(top_k) < k:
    top_k.extend(top_k[:(k - len(top_k))])

  return top_k[:k]
  
def get_top_k_pos_neg(row, k, txt_database):
  ctxs = row["ctxs"]
  top_k_pos = []
  top_k_neg = []
  for ctx in ctxs:
    if ctx["has_answer"] and len(top_k_pos) < k:
      text = txt_database[ctx["id"] - 1]["text"]
      top_k_pos.append((text, "true"))
    if not ctx["has_answer"] and len(top_k_neg) < k: 
      text = txt_database[ctx["id"] - 1]["text"]
      top_k_neg.append((text, "false"))
    if len(top_k_pos) == k and len(top_k_neg):
      break 
  if len(top_k_pos) == 0: 
    return []
  while len(top_k_pos) < k:
    top_k_pos.extend(top_k_pos[:(k - len(top_k_pos))])
  while len(top_k_neg) < k:
    top_k_neg.extend(top_k_neg[:(k - len(top_k_neg))])
  top_k = []
  for i in range(k):
    top_k.append(top_k_pos[i])
    top_k.append(top_k_neg[i])
  return top_k


In [12]:
def create_pos_txt_col(example, k, txt_database):
  return {"pos_text": get_top_k_pos(example, k, txt_database)}

def create_pos_neg_txt_col(example, k, txt_database):
  return {"pos_neg_text": get_top_k_pos_neg(example, k, txt_database)}


In [13]:
from functools import partial
nq_open["train"] = nq_open["train"].map(partial(create_pos_txt_col, k=20, txt_database=wikipedia_txt["train"]), num_proc=4)

#3:   0%|          | 0/19792 [00:00<?, ?ex/s]

#2:   0%|          | 0/19792 [00:00<?, ?ex/s]

#0:   0%|          | 0/19792 [00:00<?, ?ex/s]

#1:   0%|          | 0/19792 [00:00<?, ?ex/s]

In [14]:
nq_open["train"] = nq_open["train"].map(partial(create_pos_neg_txt_col, k=10, txt_database=wikipedia_txt["train"]), num_proc=4)

#0:   0%|          | 0/19792 [00:00<?, ?ex/s]

#1:   0%|          | 0/19792 [00:00<?, ?ex/s]

#3:   0%|          | 0/19792 [00:00<?, ?ex/s]

#2:   0%|          | 0/19792 [00:00<?, ?ex/s]

In [15]:
nq_open["validation"] = nq_open["validation"].map(partial(create_pos_txt_col, k=20, txt_database=wikipedia_txt["train"]), num_proc=4)
nq_open["validation"] = nq_open["validation"].map(partial(create_pos_txt_col, k=20, txt_database=wikipedia_txt["train"]), num_proc=4)

#0:   0%|          | 0/2190 [00:00<?, ?ex/s]

#1:   0%|          | 0/2189 [00:00<?, ?ex/s]

#3:   0%|          | 0/2189 [00:00<?, ?ex/s]

#2:   0%|          | 0/2189 [00:00<?, ?ex/s]

#0:   0%|          | 0/2190 [00:00<?, ?ex/s]

#1:   0%|          | 0/2189 [00:00<?, ?ex/s]

#3:   0%|          | 0/2189 [00:00<?, ?ex/s]

#2:   0%|          | 0/2189 [00:00<?, ?ex/s]

In [16]:
from tqdm import tqdm
def create_ranking_loss_baseline_examples(dataset, n=None):
  if not n:
    n = len(dataset)
  with_answer = 0 
  inputs = []
  targets = []
  for i in tqdm(range(n)):
    texts = dataset[i]["pos_neg_text"]
    if not texts: continue 
    question = dataset[i]["question"]
    current_inputs = [f"Question: {question} Passage: {text[0]} Relevant: " for text in texts]
    current_targets = [text[1] for text in texts]
    inputs.extend(current_inputs)
    targets.extend(current_targets)
    with_answer += 1 
  k = [len(targets)/(with_answer * 2)] * len(targets)
  return {"inputs": inputs, "targets": targets, "k_pos_neg": k}

def create_q_gen_baseline_examples(dataset, n=None):
  if not n:
    n = len(dataset)
  inputs = []
  targets = []
  with_answer = 0 
  for i in tqdm(range(n)):
    texts = dataset[i]["pos_text"]
    if not texts: continue 
    question = dataset[i]["question"]
    current_inputs = [f"Passage: {text[0]} Please write a question based on this passage" for text in texts]
    current_targets = [question for text in texts]
    inputs.extend(current_inputs)
    targets.extend(current_targets)
    with_answer += 1     
  k = [len(targets)/(with_answer)] * len(targets)
  return {"inputs": inputs, "targets": targets, "k_pos": k}

def create_q_gen_ranking_baseline_examples(dataset, n=None):
  if not n:
    n = len(dataset)
  inputs = []
  targets = []
  with_answer = 0 
  for i in tqdm(range(n)):
    texts = dataset[i]["pos_neg_text"]
    if not texts: continue 
    question = dataset[i]["question"]
    current_inputs = [f"Passage: {text[0]} Please write a question based on this passage" for text in texts]
    current_targets = [question for text in texts]
    inputs.extend(current_inputs)
    targets.extend(current_targets)
    with_answer += 1   
  k = [len(targets)/(with_answer * 2)] * len(targets)
  return {"inputs": inputs, "targets": targets, "k_pos_neg": k}



In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5ForConditionalGeneration


model_checkpoint = "google/t5-base-lm-adapt"
model_checkpoint_2 = "bigscience/T0_3B"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [18]:
def preprocess_function(examples, max_input_length, max_target_length, input_col):
    model_inputs = tokenizer(
        examples[input_col],
        max_length=max_input_length,
        truncation=True,  padding="longest"
    )
    labels = tokenizer(text_target=examples["targets"], max_length=max_target_length, truncation=True,  padding="longest", return_tensors="pt"
    )
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["decoder_attention_mask"] = labels["attention_mask"]
    return model_inputs

In [19]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/655 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [20]:
def qg_batching(question, ctxs, has_ans, evidence_txts):
  texts = [evidence_txts[ctx["id"] - 1]["text"] for ctx in ctxs]
  texts = [f"Passage: {text} Please write a question based on this passage" for text in texts]
  targets = [question for text in texts]
  eval_dataset = datasets.Dataset.from_dict({'inputs': texts, 'targets': targets})  
  return eval_dataset

def relevance_batching(question, ctxs, has_ans, evidence_txts):
  texts = [evidence_txts[ctx["id"] - 1]["text"] for ctx in ctxs]
  texts = [f"Question: {question} Passage: {text} Relevant: " for text in texts]
  targets = ["true" if ans else "false" for ans in has_ans]
  new_dataset = datasets.Dataset.from_dict({'inputs': texts, 'targets': targets})
  return new_dataset
  
def qg_ranking(logits, labels):
  log_softmax = torch.nn.LogSoftmax(dim=-1)
  log_soft = log_softmax(logits)
  labels = labels.unsqueeze(2)
  log_soft = log_soft.gather(2, labels).squeeze(2)
  log_soft = log_soft.mean(dim=1)
  return log_soft
  
def relevance_ranking(logits, labels):
  softmax = torch.nn.Softmax(dim=-1)
  probs = softmax(logits)
  labels = labels.unsqueeze(2)
  probs = probs.gather(2, labels).squeeze(2)[:, 0]
  return probs




In [21]:
from transformers.utils.logging import disable_progress_bar
from transformers import DataCollatorForSeq2Seq
from torch.utils.data import SequentialSampler, DataLoader
import numpy as np
def evaluate_recall(validation, k, model, tokenizer, batch_size, evidence_txts, 
                       preprocess_function, isRanking=False, isQG=True):
  
  assert k // batch_size != 0, "k must be multiple of batch_size"
  assert batch_size // 2 != 0, "Batch Size Must Be Even"

  if k < batch_size: 
    batch_size = k

  original_recall = []
  current_recall = []
  device = "cuda:0" if torch.cuda.is_available() else "cpu"
  model.to(device)
  losses = []
  for i in tqdm(range(len(validation))):

    # Extract Question, Passages, and Info on Whether Passages have Answer
    question = validation[i]["question"]
    ctxs = validation[i]["ctxs"][:k]
    has_ans = [ctx["has_answer"] for ctx in ctxs]
    has_ans = torch.BoolTensor(has_ans)

    # Build Data as Model Expects
    if isQG: 
      eval_dataset = qg_batching(question, ctxs, has_ans, evidence_txts)
    else: 
      eval_dataset = relevance_batching(question, ctxs, has_ans, evidence_txts)

    datasets.utils.disable_progress_bar()
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    eval_dataset = eval_dataset.map(partial(preprocess_function, max_input_length=300,
                                                            max_target_length=50, input_col='inputs'), 
                                    batched=True)

    eval_dataset = eval_dataset.remove_columns(["inputs", "targets"])
    eval_dataset.set_format(type="torch")
    eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=data_collator, shuffle=False)
    
    # Calculate Log Scores and Get Ranking
    scores = []
    model.eval()
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
          batch = {k: v.to(device) for k, v in batch.items()}
          outputs = model(**batch)
          logits = outputs.logits
          labels = batch["labels"]
          if isRanking:
            loss = ranking_loss(logits, batch["labels"], .1 * outputs.loss.item(), 
                                batch["labels"].size(dim=0))
            losses.append(loss)
          else: 
            losses.append(outputs.loss)
          if isQG: 
            score = qg_ranking(logits, labels)
          else: 
            score = relevance_ranking(logits, labels)
          scores.append(score)
    
    scores = torch.cat(scores)
    topk_scores, indexes = torch.topk(scores, k=len(scores))

    # Collect Stats for Recall
    ranked_answers = has_ans[indexes]
    current_has_ans = torch.cumsum(ranked_answers, dim=0) > 0
    original_has_ans = torch.cumsum(has_ans, dim=0) > 0

    original_recall.append(original_has_ans.tolist())
    current_recall.append(current_has_ans.tolist())

  original_recall = np.mean(np.array(original_recall), axis=0)
  current_recall = np.mean(np.array(current_recall), axis=0)
  loss = sum(losses) / len(losses)
  return original_recall, current_recall, loss



    



In [ ]:
val_dataset = nq_open["train"].select(range(30,50))
original_recall, current_recall, loss = evaluate_recall(val_dataset, k=30, model=model,
                                                     tokenizer=tokenizer, 
                                                     batch_size=10, 
                                                     evidence_txts = wikipedia_txt["train"], 
                                                     preprocess_function=preprocess_function)


In [ ]:
!nvidia-smi

In [63]:
def ranking_loss(outputs, labels, margin, batch_size):
  assert batch_size // 2 != 0, "Batch Size must be even" 

  log_softmax = torch.nn.LogSoftmax(dim=-1)
  outputs = log_softmax(outputs)

  ce_loss = torch.nn.CrossEntropyLoss()
  pos_outputs = outputs[::2, :, :]
  neg_outputs = outputs[1::2, :, :]
  flat_size = pos_outputs.size(-1)
  pos_loss = ce_loss(pos_outputs.reshape(-1, flat_size), labels[::2, :].reshape(-1))
  neg_loss = ce_loss(neg_outputs.reshape(-1, flat_size), labels[1::2, :].reshape(-1))

  margin_loss = torch.nn.MarginRankingLoss(margin)
  loss = margin_loss(pos_loss, neg_loss, torch.tensor(-1))

  return loss

In [67]:
from transformers import AdamW
from transformers import get_scheduler
train_dataset = datasets.Dataset.from_dict(create_q_gen_ranking_baseline_examples(nq_open["train"].select(range(100, 500))))
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
train_dataset = train_dataset.map(partial(preprocess_function, max_input_length=300,
                                                            max_target_length=50, input_col='inputs'), 
                                    batched=True)

train_dataset = train_dataset.remove_columns(["inputs", "targets", "k_pos_neg"])
train_dataset.set_format(type="torch")
batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=data_collator, shuffle=False)

optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
model.train()
progress_bar = tqdm(range(num_training_steps))
device = "cuda:0"
model.to(device)
for epoch in range(num_epochs):
  losses = []
  for step, batch in enumerate(train_dataloader):
          batch = {k: v.to(device) for k, v in batch.items()}
          outputs = model(**batch)
          logits = outputs.logits
          loss = ranking_loss(logits, batch["labels"], .1 * outputs.loss.item(), 
                                batch["labels"].size(dim=0))
          losses.append(loss)
          loss.backward()

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
  print(f'Train Loss in Epoch {epoch}: {sum(losses)/len(losses)}')
  val_dataset = nq_open["validation"].select(range(30,50))
  original_recall, current_recall, loss = evaluate_recall(val_dataset, k=30, model=model,
                                                     tokenizer=tokenizer, 
                                                     batch_size=10, 
                                                     evidence_txts = wikipedia_txt["train"], 
                                                     preprocess_function=preprocess_function, isRanking=True)
  print(f'DPR Recall @ 5: {original_recall[5]}')
  print(f'Our Recall @ 5: {current_recall[5]}')
  print(f'Validation Loss: {loss.item()}')
  
  


100%|██████████| 400/400 [00:04<00:00, 85.77it/s]


  0%|          | 0/8 [00:00<?, ?ba/s]

 33%|███▎      | 1850/5550 [10:18<18:37,  3.31it/s]

Train Loss in Epoch 0: 1.0772570371627808



100%|██████████| 20/20 [00:10<00:00,  1.84it/s]


...

DPR Recall @ 5: 0.65
Our Recall @ 5: 0.65
Validation Loss: 0.7033696174621582


100%|██████████| 5550/5550 [30:35<00:00,  3.47it/s]

Train Loss in Epoch 2: 0.08343952149152756



100%|██████████| 20/20 [00:10<00:00,  1.85it/s]

DPR Recall @ 5: 0.65
Our Recall @ 5: 0.6
Validation Loss: 0.7212198972702026


In [ ]:
original_recall, current_recall, loss = evaluate_questions(nq_open["train"].select(range(100, 500)), k=30, model=model,
                                                     tokenizer=tokenizer, 
                                                     batch_size=10, 
                                                     evidence_txts = wikipedia_txt["train"], 
                                                     preprocess_function=preprocess_function)
print(original_recall, current_recall, loss)

In [ ]:
print(current_recall[19], original_recall[19])

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
original_recall, current_recall, loss = evaluate_recall(nq_open["train"].select(range(100, 110)), k=30, model=model,
                                                     tokenizer=tokenizer, 
                                                     batch_size=10, 
                                                     evidence_txts = wikipedia_txt["train"], 
                                                     preprocess_function=preprocess_function, isQG=False)

print(current_recall[19], original_recall[19])

In [ ]:
print(current_recall[19], original_recall[19], loss)

In [ ]:
from transformers import AdamW
from transformers import get_scheduler
train_dataset = datasets.Dataset.from_dict(create_ranking_loss_baseline_examples(nq_open["train"].select(range(100, 500))))
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
train_dataset = train_dataset.map(partial(preprocess_function, max_input_length=300,
                                                            max_target_length=50, input_col='inputs'), 
                                    batched=True)

train_dataset = train_dataset.remove_columns(["inputs", "targets", "k_pos_neg"])
train_dataset.set_format(type="torch")
batch_size = 10
train_dataloader = DataLoader(train_dataset, batch_size=10, collate_fn=data_collator, shuffle=False)

optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
model.train()
progress_bar = tqdm(range(num_training_steps))
device = "cuda:0"
model.to(device)
for epoch in range(num_epochs):
  losses = []
  for step, batch in enumerate(train_dataloader):
          batch = {k: v.to(device) for k, v in batch.items()}
          outputs = model(**batch)
          loss = outputs.loss
          losses.append(loss)
          loss.backward()

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)

  print(f'Train Loss in Epoch {epoch}: {sum(losses)/len(losses)}')
  val_dataset = nq_open["validation"].select(range(30,50))
  original_recall, current_recall, loss = evaluate_recall(val_dataset, k=30, model=model,
                                                     tokenizer=tokenizer, 
                                                     batch_size=10, 
                                                     evidence_txts = wikipedia_txt["train"], 
                                                     preprocess_function=preprocess_function, isQG=False)
  print(f'DPR Recall @ 5: {original_recall[5]}')
  print(f'Our Recall @ 5: {current_recall[5]}')
  print(f'Validation Loss: {loss.item()}')
  
  

In [ ]:
original_recall